### Tarefas do dia 
1. Rafaça o código de treinamento da aula 12: 
    - “K-Nearest Neighbors - Prática” no seu computador usando o Jupyter Notebook ou o Google Colabs.
2. Retreino o algoritmo com os seguintes valores para
    - K: [3, 5, 7, 9, 11, 13, 15, 17, 19 e 21] e anote a acurácia.
3. Qual o problema principal de usar a métrica acurácia? 
    - Escreve um exemplo hipotético, no qual o problema acontece.
   R: O problema principal de usar a métrica acurácia é que ela pode ser enganosa em situações em que as
    classes estão desbalanceadas. A acurácia é calculada como a proporção de amostras corretamente classificadas
    em relação ao total de amostras. No entanto, em um cenário com classes desbalanceadas, em que uma classe
    é muito mais frequente do que a outra, um classificador pode ter uma alta acurácia simplesmente classificando
    todas as instâncias como pertencentes à classe majoritária. Isso ocorre porque a maioria das previsões
    estará correta para a classe majoritária, mas muito poucas previsões serão corretas para a classe minoritária.
    Portanto, a acurácia pode não fornecer uma visão precisa do desempenho do classificador.

   Exemplo hipotético: Considere um conjunto de dados em que 90% das amostras pertencem à classe A
   e 10% pertencem à classe B. Um classificador que sempre prevê a classe A teria uma acurácia de 90%. 
   No entanto, isso não significa que o classificador seja bom na detecção da classe B, 
   que pode ser mais importante ou relevante para o problema em questão.   

4. Explique com um pequeno texto ilustrando a diferença entre:
    - métrica de Precision e Recall e mostrando quando usa deve ser escolhida em relação a outra.

R: A métrica de Precision (precisão) e Recall (revocação) são métricas utilizadas na avaliação de classificadores binários.
A precisão é a proporção de verdadeiros positivos (TP) em relação ao total de instâncias previstas como positivas, ou seja,
ela mede a precisão das previsões positivas. A fórmula para calcular a precisão é: Precision = TP / (TP + FP)

R: O recall é a proporção de verdadeiros positivos em relação ao total de instâncias verdadeiramente positivas, ou seja,
mede a capacidade do classificador de encontrar todas as instâncias positivas. A fórmula para calcular o recall é: Recall = TP / (TP + FN)

A escolha entre precisão e recall depende do contexto do problema e das necessidades específicas do usuário.

Precision é útil quando o foco principal é minimizar os falsos positivos, ou seja, evitar prever erroneamente uma instância como positiva
quando ela é na verdade negativa. Por exemplo, em um sistema de detecção de spam de e-mails, é importante ter uma alta precisão para garantir
que poucos e-mails legítimos sejam classificados como spam.

Recall é útil quando o objetivo principal é minimizar os falsos negativos, ou seja, evitar que instâncias positivas sejam erroneamente previstas
como negativas. Por exemplo, em um sistema de diagnóstico médico, é crucial ter um alto recall para garantir que poucas condições médicas graves
sejam negligenciadas.

5. Escreve um trecho de código que automatize o treinamento do algoritmo
    - K-NN, a fim de encontrar o melhor valor para K, do exercício 2.

6. Escreva um pequeno texto, explicando as 6 denominações da
    - matriz de confusão: P, N, TP, FN, FP e TN

7. No conjunto de dados usado na aula 12: “K-Nearest Neighbors - Prática”, existe alguma variável que fere as premissas do K-NN?
   Se sim ou não, explique.
R: sim tem duas variaveis object e umas das premissas do knn e que sejam variaveis numericas.    
    As premissas do algoritmo K-NN (K-Vizinhos Mais Próximos) são:
Variáveis numéricas contínuas: O K-NN é adequado para lidar com variáveis numéricas contínuas. 
Ele utiliza a distância euclidiana ou outras métricas de distância para calcular a proximidade entre instâncias.
Espaço Euclidiano: O K-NN assume que os dados estão em um espaço euclidiano, ou seja, as distâncias entre as instâncias
são medidas usando a geometria euclidiana. Portanto, é importante garantir que as variáveis sejam numericamente comparáveis
 e que a distância euclidiana seja uma métrica válida para medir a semelhança entre as instâncias.
Instâncias próximas são semelhantes: O K-NN pressupõe que instâncias semelhantes tendem a estar próximas umas das outras
no espaço de recursos.Isso significa que, para que o K-NN funcione corretamente, é necessário que as amostras do mesmo rótulo 
estejam agrupadas e separadas de outras classes.
Conjunto de treinamento representativo: O K-NN assume que o conjunto de treinamento é uma representação adequada da população geral.
Isso implica que o conjunto de treinamento deve abranger adequadamente a variabilidade dos dados e capturar todas as possíveis combinações
de características que ocorrerão durante o teste.
É importante ter em mente essas premissas ao aplicar o algoritmo K-NN, pois violações dessas premissas podem afetar o desempenho e a eficácia
do modelo. Caso as premissas não sejam atendidas, pode ser necessário considerar outros algoritmos de classificação mais adequados para o conjunto
de dados em questão.

8. Faça a seguinte bateria de testes

8.1 Class balanceada originalmente:

    1. Faça a matriz de confusão, calcule a acurácia, recall e precision do conjunto de dados original

    2. Anote os resultados. 

8.2 Classe balanceada:
    1. Mantenha a proporção de 50% das linhas da planilha de dados com exemplos da classe “Conceder” e 50% com a classe “Negar”.

    2. Faça a matriz de confusão, calcule a acurácia, recall e precision.

    3. Anote os resultados.

        
8.3 Classe desbalanceada 90/10
    1. Mantenha a proporção de 90% das linhas da planilha de dados com exemplos da classe “Conceder” e 10% com a classe “Negar”.

    2. Faça a matriz de confusão, calcule a acurácia, recall e precision.

    3. Anote os resultados.

8.4 Refaça o exercícios 8.3 com as seguintes variações: 80/20, 70/30 e 60/40

8.5 Crie um tabela de comparação entre os resultados das 3 métricas ( acurácia, recall e precision ) 
    para cada uma das proporções ( 50/50, 90/10, 80/20, 70/30, 60/40 ) e responda as seguintes perguntas:

    1. Como a métrica da acurácia se comporta com a variação do desbalanceamento do conjunto de dados?

    2. O que acontece com a métrica “Precison” e “Recall” a medida que os conjunto de dados tendem ao balanceamento de 50/50?

    3. Observando as respostas anteriores, quais são as ações que aumentam ou diminuem a métrica de “Precision” ou “Recall” 
    de um problema de negócio?
     
8.6 Escreva um artigo para blog da Comunidade DS, descrevendo o seu aprendizado em relação ao comportamento das métricas de Precision,
 Recall e Acurácia a partir dos seus experimentos com os dados desbalanceados.

Para realizar a bateria de testes proposta, é necessário ter acesso aos dados e ao algoritmo de classificação utilizado. Sem essas informações, não é possível realizar os cálculos e análises solicitados. No entanto, posso fornecer uma explicação geral sobre o comportamento das métricas de Precision, Recall e Acurácia em relação ao desbalanceamento dos dados.

O desbalanceamento de classes em um conjunto de dados ocorre quando uma classe tem um número significativamente maior ou menor de exemplos em comparação com outras classes. Isso pode afetar o desempenho das métricas de avaliação do modelo. Vamos analisar cada uma das perguntas propostas:

8.5.1. Acurácia:
A métrica de acurácia é influenciada pelo desbalanceamento dos dados. Quando as classes estão desbalanceadas, ou seja, uma classe tem uma frequência muito maior ou menor em relação à outra, o modelo pode ter um viés em direção à classe majoritária. Isso pode levar a um alto valor de acurácia, que não reflete necessariamente a capacidade do modelo em prever corretamente as instâncias da classe minoritária. Portanto, em conjuntos de dados desbalanceados, a acurácia pode ser enganosa e não fornecer uma medida precisa da qualidade do modelo.

8.5.2. Precision e Recall:
A medida que o conjunto de dados tende a um balanceamento de 50/50, espera-se que as métricas de Precision e Recall se aproximem e sejam equilibradas para ambas as classes. Precision mede a proporção de exemplos classificados corretamente como positivos em relação ao total de exemplos classificados como positivos, enquanto Recall mede a proporção de exemplos positivos corretamente classificados em relação ao total de exemplos positivos no conjunto de dados. Em um conjunto de dados desbalanceado, onde a classe minoritária é rara em relação à classe majoritária, é comum observar um alto Recall para a classe majoritária, mas um baixo Recall para a classe minoritária. Isso ocorre porque o modelo pode estar focando principalmente na classe majoritária para obter uma alta acurácia geral.

8.5.3. Ações para aumentar Precision ou Recall:
Para melhorar a métrica de Precision, é necessário focar em reduzir os falsos positivos, ou seja, os exemplos que são erroneamente classificados como positivos. Isso pode ser feito ajustando o limiar de classificação do modelo ou aplicando técnicas de ajuste de custo que penalizam mais os erros de classificação da classe positiva.

Para melhorar a métrica de Recall, é necessário focar em reduzir os falsos negativos, ou seja, os exemplos positivos que são erroneamente classificados como negativos. Isso pode ser alcançado ajustando o limiar de classificação do modelo ou aplicando técnicas de balanceamento de dados, como oversampling (gerando mais exemplos da classe minoritária) ou undersampling (reduzindo o número de exemplos da classe majoritária).

8.6. Artigo para o blog da Comunidade DS:
Como a resposta é extensa, não é possível fornecer um artigo completo neste contexto. No entanto, para escrever um artigo sobre o aprendizado em relação ao comportamento das métricas de Precision, Recall e Acurácia com dados desbalanceados, você pode seguir a estrutura abaixo:

Introdução: Apresente o problema do desbalanceamento de classes em conjuntos de dados e a importância de compreender o comportamento das métricas de avaliação.

Explicação das métricas: Descreva brevemente as métricas de Precision, Recall e Acurácia e sua relevância na avaliação de modelos de classificação.

Experimentos: Descreva os experimentos realizados com dados desbalanceados, incluindo as diferentes proporções de classes e os resultados obtidos para cada métrica.

Análise dos resultados: Analise os resultados obtidos e discuta como o desbalanceamento afeta as métricas e a interpretação dos resultados do modelo.

Ações para melhorar as métricas: Explique as possíveis ações que podem ser tomadas para melhorar as métricas de Precision e Recall em conjuntos de dados desbalanceados.

Considerações finais: Conclua o artigo resumindo as principais descobertas e destacando a importância de considerar o desbalanceamento de classes ao avaliar modelos de classificação.

Lembre-se de fornecer exemplos práticos e ilustrativos ao longo do artigo para facilitar a compreensão dos leitores.

# Import Libries

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics as mt

In [21]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score


# Load Dataset

In [2]:
# 1. Prepare os dados
df = pd.read_csv('train.csv')
df.head()

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional
0,1767,21,278.172008,2577.05,24196.89636,104.306544,31.038763,6,5,7,21,14,9,15,Não,Não,Negar
1,11920,40,268.874152,2465.39,19227.37796,69.858778,36.917093,5,8,5,40,23,10,18,Não,Não,Negar
2,8910,36,446.643127,1055.29,42822.28223,134.201478,34.561714,0,3,6,26,13,3,15,Sim,Não,Negar
3,4964,58,321.141267,703.05,51786.82600,297.350067,31.493561,0,3,7,12,7,2,1,Sim,Não,Negar
4,10100,35,428.716114,891.29,44626.85346,134.201478,28.028887,2,8,7,24,10,8,20,Sim,Não,Negar


In [3]:
df.dtypes

id_cliente                   int64
idade                        int64
saldo_atual                float64
divida_atual               float64
renda_anual                float64
valor_em_investimentos     float64
taxa_utilizacao_credito    float64
num_emprestimos              int64
num_contas_bancarias         int64
num_cartoes_credito          int64
dias_atraso_dt_venc          int64
num_pgtos_atrasados          int64
num_consultas_credito        int64
taxa_juros                   int64
investe_exterior            object
pessoa_polit_exp            object
limite_adicional            object
dtype: object

In [4]:
# Var resposta
df.loc[:,'limite_adicional'].unique()

array(['Negar', 'Conceder'], dtype=object)

In [5]:
# contagem de quantos negar/conceder
contagem = df.loc[:,'limite_adicional'].value_counts()
# Porcentagem da quantidade Negar/Conceder
porcentagem = df.loc[:,'limite_adicional'].value_counts(normalize=True)
print(contagem)
print(porcentagem)

Negar       7995
Conceder    1505
Name: limite_adicional, dtype: int64
Negar       0.841579
Conceder    0.158421
Name: limite_adicional, dtype: float64


In [6]:
# Conjunto de dados X (features) e y (rótulos/classes)
feactures = ['idade', 'saldo_atual', 'divida_atual', 'renda_anual',
       'valor_em_investimentos', 'taxa_utilizacao_credito', 'num_emprestimos',
       'num_contas_bancarias', 'num_cartoes_credito', 'dias_atraso_dt_venc',
       'num_pgtos_atrasados', 'num_consultas_credito', 'taxa_juros', 'investe_exterior', 'pessoa_polit_exp']
label = ['limite_adicional']      

In [7]:
# 2. Divida os dados em conjunto de treinamento
x_train = df.loc[:,feactures]
y_train = df.loc[:,label]

In [8]:
# Supondo que 'x_train' seja o DataFrame que contém seus dados de treinamento e devemos excluir dados nao relevantes ou que nao seja float/int
drop_n_relevante = ['investe_exterior', 'pessoa_polit_exp']
x_train = x_train.drop(drop_n_relevante, axis=1)

# Treinamento

In [16]:
# Aqui está um trecho de código que automatiza o treinamento do algoritmo K-NN 
# para encontrar o melhor valor para K, com base no exercício 2:

# 2. Defina uma lista de valores K para experimentar
k_values = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]

# 3. Crie uma instância do classificador KNN
knn = KNeighborsClassifier()

# 4. Defina os parâmetros para a busca por hiperparâmetros
param_grid = {'n_neighbors': k_values}

# # 5. Realize a busca por hiperparâmetros usando validação cruzada
# grid_search = GridSearchCV(knn, param_grid, cv=5) # cv é o número de folds na validação cruzada
# grid_search.fit(x_train, y_train.ravel())  # Usar y_train.ravel() em vez de y_train

# 5. Realize a busca por hiperparâmetros usando validação cruzada
grid_search = GridSearchCV(knn, param_grid, cv=7)
grid_search.fit(x_train, y_train.values.ravel())  # Converter y_train para um array numpy usando .values.ravel()


# 6. Obtenha os resultados da busca por hiperparâmetros
print("Melhores parâmetros: ", grid_search.best_params_)
print("Melhor acurácia média: ", grid_search.best_score_)

# Nesse código, a função GridSearchCV é utilizada para realizar uma busca exaustiva pelos melhores parâmetros. 
# O parâmetro param_grid especifica os valores que serão experimentados para o hiperparâmetro n_neighbors (valores de K). 
# O resultado da busca é armazenado em grid_search.best_params_, que contém os melhores parâmetros encontrados, e grid_search.best_score_, 
# que contém a melhor acurácia média obtida durante a validação cruzada.


Melhores parâmetros:  {'n_neighbors': 21}
Melhor acurácia média:  0.83789457102763


### As premissas do algoritmo K-NN (K-Vizinhos Mais Próximos) são:

Variáveis numéricas contínuas: O K-NN é adequado para lidar com variáveis numéricas contínuas. Ele utiliza a distância euclidiana ou outras métricas de distância para calcular a proximidade entre instâncias.
Espaço Euclidiano: O K-NN assume que os dados estão em um espaço euclidiano, ou seja, as distâncias entre as instâncias são medidas usando a geometria euclidiana. Portanto, é importante garantir que as variáveis sejam numericamente comparáveis e que a distância euclidiana seja uma métrica válida para medir a semelhança entre as instâncias.
Instâncias próximas são semelhantes: O K-NN pressupõe que instâncias semelhantes tendem a estar próximas umas das outras no espaço de recursos. Isso significa que, para que o K-NN funcione corretamente, é necessário que as amostras do mesmo rótulo estejam agrupadas e separadas de outras classes.
Conjunto de treinamento representativo: O K-NN assume que o conjunto de treinamento é uma representação adequada da população geral. Isso implica que o conjunto de treinamento deve abranger adequadamente a variabilidade dos dados e capturar todas as possíveis combinações de características que ocorrerão durante o teste.
É importante ter em mente essas premissas ao aplicar o algoritmo K-NN, pois violações dessas premissas podem afetar o desempenho e a eficácia do modelo. Caso as premissas não sejam atendidas, pode ser necessário considerar outros algoritmos de classificação mais adequados para o conjunto de dados em questão.

### Predicao

In [17]:
# 5. Faça previsões no conjunto de teste
y_pred =grid_search.predict(x_train)

In [20]:
y_pred

array(['Negar', 'Negar', 'Negar', ..., 'Negar', 'Negar', 'Negar'],
      dtype=object)

In [18]:
# copy dataframe
df_result = df.copy()# veio todas
# criar uma col  classificacao
df_result['classificacao'] = y_pred

In [19]:
df_result.head()

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional,classificacao
0,1767,21,278.172008,2577.05,24196.89636,104.306544,31.038763,6,5,7,21,14,9,15,Não,Não,Negar,Negar
1,11920,40,268.874152,2465.39,19227.37796,69.858778,36.917093,5,8,5,40,23,10,18,Não,Não,Negar,Negar
2,8910,36,446.643127,1055.29,42822.28223,134.201478,34.561714,0,3,6,26,13,3,15,Sim,Não,Negar,Negar
3,4964,58,321.141267,703.05,51786.82600,297.350067,31.493561,0,3,7,12,7,2,1,Sim,Não,Negar,Negar
4,10100,35,428.716114,891.29,44626.85346,134.201478,28.028887,2,8,7,24,10,8,20,Sim,Não,Negar,Negar


# 4.0 Performace

# Matriz da confusao

A matriz de confusão é uma tabela que resume o desempenho de um modelo de classificação em um conjunto de dados de teste. 
As seis denominações da matriz de confusão são:
P (Positivo): Representa as instâncias da classe positiva.
N (Negativo): Representa as instâncias da classe negativa.
TP (True Positive): Representa as instâncias positivas que foram corretamente classificadas como positivas.
FN (False Negative): Representa as instâncias positivas que foram incorretamente classificadas como negativas.
FP (False Positive): Representa as instâncias negativas que foram incorretamente classificadas como positivas.
TN (True Negative): Representa as instâncias negativas que foram corretamente classificadas como negativas.

Essas denominações são utilizadas para calcular várias métricas de avaliação, como precisão, revocação e acurácia, que fornecem insights sobre o desempenho do modelo de classificação.

In [30]:
df_result['acertos'] = df_result.loc[:,['id_cliente','limite_adicional','classificacao']].apply(lambda x: 1 if x['limite_adicional'] == x['classificacao'] else 0, axis=1)

In [31]:
df_result.loc[:,['id_cliente','idade','limite_adicional','classificacao','acertos'] ].sample(20)

,id_cliente,idade,limite_adicional,classificacao,acertos
2376,1567,56,Conceder,Conceder,1
6974,9859,22,Negar,Conceder,0
1725,10118,43,Negar,Negar,1
364,2471,36,Negar,Negar,1
1468,6299,52,Negar,Negar,1
8317,7872,59,Negar,Negar,1
4514,6735,34,Negar,Negar,1
6987,4417,39,Negar,Negar,1
6324,6888,56,Negar,Negar,1
6047,1952,45,Negar,Negar,1


### 4.1 Acuracia

In [32]:
# 6. Avalie o desempenho do modelo
accuracy = mt.accuracy_score(y_train, y_pred)
print("Acurácia: %.2f%%" % (accuracy * 100))

Acurácia: 84.62%


### 4.2 Precisao

In [33]:
mt.precision_score(x_train, y_train)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

### 4.3 Recall